<a href="https://colab.research.google.com/github/jusimon/AlternusVera/blob/master/Common/FakeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
googleDrive = GoogleDrive(gauth)
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!rm -rf AlternusVera
!git clone https://github.com/jusimon/AlternusVera
!cp AlternusVera/Political_Affiliation.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 157 (delta 12), reused 6 (delta 2), pack-reused 129
Receiving objects: 100% (157/157), 34.17 MiB | 9.77 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [3]:
!rm -rf AlternusVera
!git clone https://github.com/psaini79/AlternusVera
!cp AlternusVera/sensational.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 166 (delta 23), reused 25 (delta 12), pack-reused 127
Receiving objects: 100% (166/166), 98.33 MiB | 12.15 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Checking out files: 100% (12/12), done.


In [4]:
!rm -rf AlternusVera
!git clone https://github.com/viswa0531/AlternusVera
!cp AlternusVera/Biased_Factor.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 143 (delta 1), reused 0 (delta 0), pack-reused 137
Receiving objects: 100% (143/143), 10.13 MiB | 6.08 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [5]:
!rm -rf AlternusVera
!git clone https://github.com/RajiBabu/AlternusVera
!cp AlternusVera/Authenticity.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (91/91), done.


In [6]:
from Political_Affiliation import Political_Affiliation
from sensational import sensational
from Biased_Factor import BiasedDetection
from Authenticity import Authenticity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### This function calculates fakeness percentage using the weighted_percentage function passing all factors fakeness score. It calls respective factors class to get the fakeness score of individual factor

In [0]:
from gensim.models.doc2vec import Doc2Vec
def fakecheck(dataFrame):
  biasedFactor = BiasedDetection(dataFrame).predict()
  PoliticalAff = Political_Affiliation(dataFrame).predict()
  print('calling sensationalism factor:') 
  AuthenticityFactor = Authenticity(dataFrame).predict()
  Sensationalism = sensational(dataFrame).predict()
  print('Sensationalism Factor:',Sensationalism)    
  print('biased Factor:',biasedFactor)   

  print('political_affilication_factor:',PoliticalAff)
  
  print('authenticity_factor:',AuthenticityFactor)
  #  print('spelling_error_factor:',spelling_error_factor)
  #  print('spam_detection_factor:',spam_detection_factor)
  #  print('stance_detection_factor:',stance_detection_factor)
  return weighted_percentage(Sensationalism, PoliticalAff, biasedFactor, AuthenticityFactor)

In [0]:
def weighted_percentage(SensationalismF, politicalAffilitaionF, BiasedF, AuthenticityF):                       
    SensationalismF_Weigth=.84
    BiasedF_Weight=.53
    politicalAffilitaionF_Weight=.65
    AuthenticityF_Weight=.75    
    total_weight=SensationalismF_Weigth+BiasedF_Weight+politicalAffilitaionF_Weight+AuthenticityF_Weight
    print("Total Weight:", total_weight)
    return round((((SensationalismF*SensationalismF_Weigth)+
            (politicalAffilitaionF*politicalAffilitaionF_Weight)+
            (BiasedF*BiasedF_Weight)+
            (AuthenticityF*AuthenticityF_Weight))/total_weight)*100,2)

#### Check Fakeness of a news article

In [9]:
input_news='Obamacare will provide insurance to all non-U.S. residents, even if they are here illegally.A strong bipartisan majority in the House of Representatives voted to defund Obamacare.07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow.Amazon Enters eBook Subscription Market with Kindle Unlimited'
fakecheck(input_news)

calling sensationalism factor:


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/l

Sensationalism Factor: 0
biased Factor: 0
political_affilication_factor: 1
authenticity_factor: 1
Total Weight: 2.77


50.54

In [15]:
input_news='A strong bipartisan organization in the House of Representative is a must for any political ajenda to be successful - Mailer'
fakecheck(input_news)

calling sensationalism factor:


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Sensationalism Factor: 0
biased Factor: 0
political_affilication_factor: 0
authenticity_factor: 1
Total Weight: 2.77


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


27.08